# Location Extraction and Spacy Word Vectorization

In [1]:
import pandas as pd
import numpy as np
import re
import spacy
import string
import datetime

In [2]:
# Read in csv with Tweets
twitter_closures = pd.read_csv("../data/Cleaned_Tweets/cleaned_historic_official_07302019.csv")
exits = pd.read_csv("../data/interstate_exits.csv")

twitter_closures = twitter_closures[['date', 'text', 'type', 'username', 'tweet', 'state', 'road_closure']]

# Print DF shape
print(twitter_closures.shape)

# Show head 
twitter_closures.head()

(24054, 7)


date  \
0  2016-10-11 16:39:51+00:00   
1  2016-10-10 19:10:23+00:00   
2  2016-10-09 17:30:22+00:00   
3  2016-10-09 15:45:11+00:00   
4  2016-10-09 14:00:20+00:00   

                                                text      type username  \
0  the pioh for the sr 138 i-20 is going on now u...  official  GDOTATL   
1  we appreciate all the hard work our crews have...  official  GDOTATL   
2  need info on re-entry  check here  also follow...  official  GDOTATL   
3  do you need some updates  you can find all of ...  official  GDOTATL   
4  did you know that the  511georgia website and ...  official  GDOTATL   

                                               tweet    state  road_closure  
0  The PIOH for the SR 138 I-20 is going on now u...  Georgia             0  
1  We appreciate all the hard work our crews have...  Georgia             0  
2  Need info on re-entry  Check here  Also follow...  Georgia             0  
3  Do you need some updates  You can find ALL of ...  Georgia             0  
4  Did you know that the  511Georgia website and ...  Georgia             0

## SpaCy Preprocessing

In [3]:
# Create new columns to transfer modified tweet text. Five versions of tweets will be created.
twitter_closures['modified_text'] = ''
twitter_closures['location'] = ''

# Show modified DF
twitter_closures.head(2)

date  \
0  2016-10-11 16:39:51+00:00   
1  2016-10-10 19:10:23+00:00   

                                                text      type username  \
0  the pioh for the sr 138 i-20 is going on now u...  official  GDOTATL   
1  we appreciate all the hard work our crews have...  official  GDOTATL   

                                               tweet    state  road_closure  \
0  The PIOH for the SR 138 I-20 is going on now u...  Georgia             0   
1  We appreciate all the hard work our crews have...  Georgia             0   

  modified_text location  
0                         
1

In [4]:
format_dict = {"hwy": "highway ",
            "blvd": "boulevard",
            " st": "street",
           "CR ": "County Road ",
           "SR ": "State Road",
           "I-": "Interstate ",
           "EB ": "Eastbound ",
           "WB ": "Westbound ",
           "SB ": "Southbound",
           "NB ": "Northbound",
           " on ": " at ",
           " E ": " East ",
           " W ": " West ",
           " S ": " South",
           " N ": " North",
           "mi ": "mile ",
           "between ": "at ",
           "Between ": "at ",
           " In ": " in",
           " in ": " at "}

In [5]:
def spacy_cleaner(df, col, word_dict):
    modified_text = "At " + df[col].replace(word_dict, regex=True)
    modified_text = modified_text.str.title()
    return modified_text

In [6]:
# run the text cleaning function and test results
twitter_closures['modified_text'] = spacy_cleaner(twitter_closures, 'tweet', format_dict)

In [7]:
twitter_closures['username'].unique()

array(['GDOTATL', 'SCDOTMidlands', 'SCDOTPeeDee', 'SCDOTLowCountry',
       'SCDOTPiedmont', '511statewideva', 'fl511_panhandl', '511Georgia',
       'fl511_state', 'fl511_northeast', 'fl511_southeast',
       'fl511_southwest', 'fl511_tampabay', 'fl511_central',
       '511centralva', '511hamptonroads', '511northernva',
       'NCDOT_Westmtn', 'NCDOT_Triangle', 'NCDOT_Piedmont',
       'NCDOT_Charlotte', 'NCDOT_Asheville', 'NCDOT_Scoast',
       'NCDOT_Ncoast'], dtype=object)

In [8]:
# convert date column to datetime
twitter_closures['date'] = pd.to_datetime(twitter_closures['date'])

In [9]:
# for ease of use, only take tweets that happened on october 4, the day Hurricane Matthew hit Jacksonville
twitter_closures = twitter_closures[(twitter_closures['date'] > '2016-10-6') & (twitter_closures['date'] < '2016-10-9')]

In [10]:
# only use tweets that contain road closures are from 'fl511 northeast'
test_df = twitter_closures[(twitter_closures['road_closure'] == 1) & (twitter_closures['username'] == 'fl511_northeast')]
test_df.shape

(267, 9)

In [11]:
def get_loc(df, text_column, location_column):
    
    # Use Spacy to extract location names from `text` column
    for i in range(len(df)):
        
        #instantiate spacy model
        nlp = spacy.load("en_core_web_sm")
        
        # create documewnt from modified text column
        doc = nlp(df[text_column].iloc[i])
        
        locations = set()
        
        # loop through every entity in the doc
        for ent in doc.ents:
            
            # find entities labelled as places
            if (ent.label_=='GPE') or (ent.label_=='FAC') or (ent.label_ == 'LOC'):
                
                # put locations in a set
                locations.add(ent.text)
                df[location_column].iloc[i] = locations
                
    return df[location_column]

In [12]:
test = get_loc(test_df, 'modified_text', 'location')

C:\Users\templ\Anaconda3\envs\dsi\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\templ\Anaconda3\envs\dsi\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
test_df['location'] = test
test_df.head()

C:\Users\templ\Anaconda3\envs\dsi\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


date  \
13806 2016-10-08 23:33:38+00:00   
13807 2016-10-08 23:18:05+00:00   
13808 2016-10-08 21:58:06+00:00   
13809 2016-10-08 21:33:05+00:00   
13810 2016-10-08 21:28:40+00:00   

                                                    text      type  \
13806  new  disabled vehicle in duval on sr-202   but...  official   
13807  new  disabled vehicle in duval on i-295 w nort...  official   
13808  new  unconfirmed disabled vehicle in duval on ...  official   
13809  update  disabled vehicle in duval on i-295 w n...  official   
13810  new  disabled vehicle in duval on i-295 w nort...  official   

              username                                              tweet  \
13806  fl511_northeast  NEW  Disabled vehicle in Duval on SR-202   But...   
13807  fl511_northeast  NEW  Disabled vehicle in Duval on I-295 W nort...   
13808  fl511_northeast  NEW  Unconfirmed disabled vehicle in Duval on ...   
13809  fl511_northeast  UPDATE  Disabled vehicle in Duval on I-295 W n...   
13810  fl511_northeast  NEW  Disabled vehicle in Duval on I-295 W nort...   

         state  road_closure  \
13806  Florida             1   
13807  Florida             1   
13808  Florida             1   
13809  Florida             1   
13810  Florida             1   

                                           modified_text  \
13806  At New  Disabled Vehicle At Duval At Sr-202   ...   
13807  At New  Disabled Vehicle At Duval At Interstat...   
13808  At New  Unconfirmed Disabled Vehicle At Duval ...   
13809  At Update  Disabled Vehicle At Duval At Inters...   
13810  At New  Disabled Vehicle At Duval At Interstat...   

                            location  
13806                                 
13807  {San Jose Right Lane Blocked}  
13808                {Interstate 95}  
13809                                 
13810

In [14]:
test_df.to_csv("../data/Loc_Extracted/tweet_locations_sample_07302019.csv", index = False)

In [15]:
test_df = pd.read_csv("../data/Loc_Extracted/tweet_locations_sample_07302019.csv")

In [16]:
# function to extract interstate, exit number, and direction
def exit_extractor (df, col, i_df):
    
    # instantiate lists for exit data
    exits = []
    interstates = []
    lat = []
    long = []
    direction = []
    
    # loop through text column
    for item in df[col]:
        
        # look for "interstate" in text
        if 'Interstate' in item:
            
            # use regex to extract interstate and number from text
            i_string = re.search(r'Interstate (\S+)', item)
            interstates.append(i_string.group(0))
            
            # use regex to extract direction following "interstate"
            d_string = re.search("(i-\d*|Interstate \d*) (South|North|East|West)*", item)
            d_string = d_string.group(0)
            d_string =  re.search("South|North|East|West", d_string)
            
            # try to extract the direction from the regex object
            # append null if an error is thrown
            try:
                d_string = d_string.group(0)
                
            except AttributeError: 
                d_string = np.nan
                
            # append direction to list    
            direction.append(d_string)
                             
            # find "exit" in text
            if 'Exit' in item:
                
                # use regex to extract interstate and number from text
                e_string = re.search(r'Exit (\S+)', item)
                exits.append(e_string.group(0))
            
            # add "none" when no exit is found   
            else:
                exits.append("None")
                
        # add "none" to exits and interstates if no interstate is found
        else:
            interstates.append("None")
            exits.append("None")
            direction.append("None")
            
    # create a new dataframe from the interstate and exit lists
    new_df = pd.DataFrame(data = interstates, columns = ['interstate'])
    new_df['exits'] = exits
    new_df['direction'] = direction
    
    # return new dataframe
    return new_df

In [17]:
# function to extract longitude and latitude, if available
def loc_extractor(new_df, i_df):
    
    lat = []
    long = []
    
    # loop through the new dataframe
    for index, row in new_df.iterrows():
        
        # find rows that have both an interstate and exit extracted
        if (row['interstate'] != "None") and (row['exits'] != "None") and row['direction'] != "None":
            
            # attempt to add lat and long based on exit and interstate strings
            try:    
                mask = (i_df['interstate'].str.contains(row['interstate'])) & (i_df['exits'].str.contains(row['exits']))
                
                # add lat and long to list
                lat.append(i_df[mask].iloc[0]['lat'])
                long.append(i_df[mask].iloc[0]['long'])
            
            # if an error occurs, append null to lat and long
            # print index where error occured
            except:
                print(f"No exit found at {index}")

                lat.append(np.nan)
                long.append(np.nan)
        # if no exit is found, add null values to lat and long
        else:
            lat.append(np.nan)
            long.append(np.nan)
            
    # add lat and long to new dataframe
    new_df['lat'] = lat
    new_df['long'] = long
    
    return new_df

In [18]:
e_df = exit_extractor(test_df, 'modified_text', exits)
e_df['direction'].value_counts()

West     98
None     68
East     36
North    33
South    32
Name: direction, dtype: int64

In [19]:
e_df = loc_extractor(exit_extractor(test_df, 'modified_text', exits), exits)

No exit found at 148
No exit found at 154
No exit found at 206
No exit found at 216
No exit found at 237
No exit found at 248
No exit found at 250


In [20]:
final_df = pd.concat([test_df, e_df], axis = 1)

In [21]:
final_df.dropna()

date  \
6    2016-10-08 20:38:21+00:00   
13   2016-10-08 19:08:32+00:00   
25   2016-10-08 15:53:25+00:00   
27   2016-10-08 15:43:35+00:00   
38   2016-10-08 13:48:03+00:00   
45   2016-10-08 11:28:02+00:00   
46   2016-10-08 11:23:02+00:00   
52   2016-10-08 06:38:01+00:00   
54   2016-10-08 05:18:27+00:00   
69   2016-10-08 03:33:06+00:00   
73   2016-10-08 02:08:00+00:00   
75   2016-10-08 00:48:01+00:00   
76   2016-10-07 23:53:02+00:00   
80   2016-10-07 23:13:02+00:00   
81   2016-10-07 23:13:02+00:00   
82   2016-10-07 23:08:01+00:00   
83   2016-10-07 22:58:03+00:00   
87   2016-10-07 20:28:00+00:00   
90   2016-10-07 19:38:00+00:00   
111  2016-10-07 14:57:59+00:00   
116  2016-10-07 11:47:58+00:00   
117  2016-10-07 10:13:01+00:00   
121  2016-10-07 05:17:55+00:00   
131  2016-10-07 00:27:54+00:00   
134  2016-10-07 00:12:54+00:00   
135  2016-10-07 00:07:57+00:00   
138  2016-10-07 00:07:55+00:00   
140  2016-10-06 23:58:06+00:00   
144  2016-10-06 23:43:16+00:00   
146  2016-10-06 23:28:05+00:00   
..                         ...   
181  2016-10-06 18:32:52+00:00   
189  2016-10-06 17:58:13+00:00   
191  2016-10-06 17:28:03+00:00   
194  2016-10-06 17:07:53+00:00   
197  2016-10-06 16:53:38+00:00   
199  2016-10-06 16:47:52+00:00   
201  2016-10-06 16:13:05+00:00   
207  2016-10-06 16:07:57+00:00   
208  2016-10-06 15:57:52+00:00   
211  2016-10-06 15:52:52+00:00   
212  2016-10-06 15:43:13+00:00   
213  2016-10-06 15:37:53+00:00   
215  2016-10-06 15:22:51+00:00   
220  2016-10-06 14:42:51+00:00   
221  2016-10-06 14:32:51+00:00   
228  2016-10-06 13:32:52+00:00   
230  2016-10-06 13:02:57+00:00   
231  2016-10-06 12:52:58+00:00   
240  2016-10-06 11:23:01+00:00   
241  2016-10-06 11:17:51+00:00   
242  2016-10-06 10:57:59+00:00   
244  2016-10-06 10:57:57+00:00   
246  2016-10-06 10:53:07+00:00   
249  2016-10-06 06:38:01+00:00   
251  2016-10-06 04:32:49+00:00   
254  2016-10-06 03:27:49+00:00   
257  2016-10-06 03:07:59+00:00   
260  2016-10-06 02:02:59+00:00   
263  2016-10-06 00:57:48+00:00   
264  2016-10-06 00:37:53+00:00   

                                                  text      type  \
6    new  disabled vehicle in duval on i-295 e nort...  official   
13   new  disabled vehicle in duval on i-295 w sout...  official   
25   new  disabled vehicle in duval on i-295 e nort...  official   
27   new  disabled vehicle in duval on i-10 east ra...  official   
38   new  disabled vehicle in duval on i-95 north r...  official   
45   update  disabled truck in duval on i-10 east r...  official   
46   new  unconfirmed disabled vehicle in duval on ...  official   
52   update  object on roadway in duval on i-95 nor...  official   
54   new  object on roadway in duval on i-95 north ...  official   
69   update  object on roadway in duval on i-295 w ...  official   
73   update  object on roadway in duval on i-295 w ...  official   
75   new  object on roadway in duval on i-295 w nor...  official   
76   new  unconfirmed traffic congestion in duval o...  official   
80   new  object on roadway in duval on i-295 w sou...  official   
81   update  object on roadway in duval on i-295 w ...  official   
82   update  object on roadway in duval on i-295 w ...  official   
83   new  unconfirmed object on roadway in duval on...  official   
87   update  object on roadway in duval on i-295 w ...  official   
90   new  object on roadway in duval on i-295 w sou...  official   
111  new  disabled truck in duval on i-95 north ram...  official   
116  update  crash in duval on i-10 west ramp to ex...  official   
117  update  crash in duval on i-95 south ramp to e...  official   
121  update  disabled vehicle in duval on i-295 w n...  official   
131  update  disabled vehicle in suwannee on i-10 w...  official   
134  update  traffic congestion in suwannee on i-10...  official   
135  new  traffic congestion in columbia on i-10 we...  official   
138  update  traffic congestion in columbia on i-10...  official   
140  update  

In [22]:
final_df.to_csv("../data/Loc_Extracted/tweet_locations_sample_07312019.csv", index = False)